In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import requests
from io import BytesIO

Process the raw excel file

In [ ]:
# Read the excel sheet and skip blank rows
raw_excel = pd.read_excel("data/DSHA LIHTC List_MAPPING.xlsx", engine='openpyxl', skiprows=[2,3], skipfooter=4, dtype=str)
# Add additional column information from the first row
raw_excel.columns = (raw_excel.columns.astype(str) + " " + raw_excel.head(1).fillna("").astype(str)).iloc[0].str.strip().values
raw_excel.rename(columns={"ALLOCATION .1 DATE": "ALLOCATION DATE", "ALLOCATION  AMOUNT": "ALLOCATION AMOUNT", "Type of Property*": "Type of Property"}, inplace=True)
raw_excel.drop(0, inplace=True)

In [ ]:
# Find and label the rows about tax year
raw_excel["is tax"] = raw_excel["PROJECT NAME & ADDRESS"].str.contains("TAX CREDIT ALLOCATIONS")

# Function that maps the boolean column "is tax", which is true when a row contains tax year information, to an integer equal to the tax year
def assign_to_year(x, i):
    # if the row is a tax year, increment i
    if x:
        i[0] = i[0] + 1
    # return an integer equal to the tax year
    return i[0] + 2016

# add a column for the tax year
index = [-1]
raw_excel["Tax Allocation Year"] = raw_excel["is tax"].apply(assign_to_year, args=[index])

# drop rows of tax year information and reformat
raw_excel = raw_excel.loc[~raw_excel["is tax"]].drop(columns="is tax").reset_index(drop=True)


In [ ]:
# Add a column that labels the primary line for an entry
raw_excel["primary"] = ~raw_excel["County"].isna()

# Function that maps each the boolean column "primary", which is true when a row contains the primary info from the dataset, to an integer that functions as an index for primary entries
def assign_to_year(x, i):
    # if the row is primary, increment i
    if x:
        i[0] = i[0] + 1
    # return an index for the primary entries
    return i[0]

# add an index column for the primary entries
index = [-1]
raw_excel["primary"] = raw_excel["primary"].apply(assign_to_year, args=[index])

In [ ]:
# Fix 'ALLOCATION AMOUNT', 'ALLOCATION DATE' swap
flipped_years = [2018, 2019, 2020, 2021, 2022]
tmp = raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years)]['ALLOCATION AMOUNT'].copy()
tmp2 = raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years)]['ALLOCATION DATE'].copy()
raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years), 'ALLOCATION AMOUNT'] = tmp2.values
raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years), 'ALLOCATION DATE'] = tmp.values


In [ ]:
# Create separate dataframes for each row in a data entry
grouped_data = raw_excel.groupby("primary")
raw_data1 = grouped_data.nth(0)
raw_data2 = grouped_data.nth(1).drop(columns="Tax Allocation Year")
raw_data3 = grouped_data.nth(2).drop(columns="Tax Allocation Year")
raw_data4 = grouped_data.nth(3).drop(columns="Tax Allocation Year")
raw_data5 = grouped_data.nth(4).drop(columns="Tax Allocation Year")

# Modify the column names for each dataframe to prepare for joining
raw_data2.columns = raw_data2.columns + " 2"
raw_data3.columns = raw_data3.columns + " 3"
raw_data4.columns = raw_data4.columns + " 4"
raw_data5.columns = raw_data5.columns + " 5"

# Join the dataframes by index and remove unused columns
flattened_data = raw_data1.join(raw_data2, how="left").join(raw_data3, how="left").join(raw_data4, how="left").join(raw_data5, how="left").dropna(axis=1, how='all').reset_index(drop=True)


In [ ]:
# Convert dates back to their orginial format
flattened_data["Placed in Service Date"] = pd.to_datetime(flattened_data["Placed in Service Date"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["ALLOCATION DATE"] = pd.to_datetime(flattened_data["ALLOCATION DATE"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Tax Credit Compliance Date"] = pd.to_datetime(flattened_data["Tax Credit Compliance Date"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Extended Use Period"] = pd.to_datetime(flattened_data["Extended Use Period"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Placed in Service Date 2"] = pd.to_datetime(flattened_data["Placed in Service Date 2"], errors='coerce').dt.strftime('%m/%d/%Y')



In [ ]:
# Combine address fields
address_columns = ["PROJECT NAME & ADDRESS", "PROJECT NAME & ADDRESS 2", "PROJECT NAME & ADDRESS 3", "PROJECT NAME & ADDRESS 4", "PROJECT NAME & ADDRESS 5"]

# Extracts addresses from projects
def extract_address(x):
    # Project 27 has three full addresses, so we use the last one listed
    if x.name == 27:
        addr = x[address_columns].dropna().values[-1]
        return addr
    # The last two lines of the address field contain the address split between two lines, except for project 27
    else:
        addr = x[address_columns].dropna().values[-2:]
        return addr[0] + ", " + addr[1]

# Extract an address for each project
flattened_data["address"] = flattened_data.apply(extract_address, axis=1)

In [ ]:
# Convert years to string
flattened_data["Tax Allocation Year"] = flattened_data["Tax Allocation Year"].astype(str)

# Print the data to a csv
flattened_data.to_csv("data/processed_data.csv", index=False)

In [ ]:
# Print the addresses to a seperate list
flattened_data["address"].to_csv("data/DSHA_addresses.csv", index=False)

At this point we transfer the address csv over to the geocoder to get the latitude and longitude of each project

In [ ]:
# Read the file of geolocated addresses
geolocations = pd.read_csv("data/counts_per_tract.csv").drop_duplicates("input addresses")
# Join the geolocations to the flattened dataframe
geolocated_data = flattened_data.merge(geolocations, left_on="address", right_on="input addresses", how="inner")
# Remove lat,lot from unsuccessfully (not in the u.s.) geolocated address
geolocated_data.loc[geolocated_data["census tract"] == "Unable To Geolocate The Address", "lot"] = np.nan
geolocated_data.loc[geolocated_data["census tract"] == "Unable To Geolocate The Address", "lat"] = np.nan

In [ ]:
# Convert lat,lot to Shapely points
geolocated_data = gpd.GeoDataFrame(geolocated_data, geometry=gpd.points_from_xy(geolocated_data['lat'], geolocated_data['lot'], crs="EPSG:4326"))


In [ ]:
"""# Visualize points on a map

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[39.74503, -75.57203], zoom_start=14, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)


# Add markers for each school
points=folium.features.GeoJson(
        geolocated_data.loc[geolocated_data["census tract"] != "Unable To Geolocate The Address"], # Full geopandas data
        control=False,
        marker = folium.CircleMarker(radius = 5, # Radius in metres
                           weight = 0, #outline weight
                           fill_color = '#d95f02', 
                           fill_opacity = 1)
        )

points.add_to(us_map)
us_map"""

In [ ]:
# extract shape files for senate districts
senate_districts2020 = gpd.read_file("data/2020Senate_Districts_Trimmed.geojson")
senate_districts2010 = gpd.read_file("data/2010Senate_Districts_Trimmed.geojson")

# Gets the senate district containing a point
def get_district(x):
    # Return a blank when an address could not be geolocated
    if x["census tract"] == "Unable To Geolocate The Address":
        return ""
    # Return the senate district containing the point otherwise
    else:
        if x["Tax Allocation Year"] < "2022":
            return senate_districts2010.loc[x["geometry"].within(senate_districts2010["geometry"])]["SLDUST"].astype(int).astype(str).values[0]
        else:
            return senate_districts2020.loc[x["geometry"].within(senate_districts2020["geometry"])]["SLDUST"].astype(int).astype(str).values[0]
# Add a column for senate district
geolocated_data["Senate District"] = geolocated_data.apply(get_district, axis=1)




In [ ]:
# Add funding source column
geolocated_data["Funding Source"] = "LIHTC"

In [ ]:
# Print the dataset with senate districts attached to a csv
geolocated_data.drop(columns=["input addresses", "census tract", "lot", "lat"]).to_file("data/DSHA_districted.geojson", driver="GeoJSON")

In [ ]:
geolocated_data.drop(columns=["input addresses", "census tract", "lot", "lat"]).columns

In [ ]:
# Count the number of Tax Credit Units in each district for each year and add them to the aggregated dataframe
geolocated_data['# of Tax Credit Units'] = geolocated_data['# of Tax Credit Units'].astype(int)
aggregated_data = geolocated_data.groupby(["Senate District", "Tax Allocation Year"]).sum()["# of Tax Credit Units"].reset_index()
aggregated_data.to_csv("data/Tax_Credit_Units_per_Senate_District.csv", index=False)





In [ ]:
# Count the Allocation Amount in each district for each year and add them to the aggregated dataframe
data_noTBD = geolocated_data.loc[geolocated_data['ALLOCATION AMOUNT'] != "TBD"]
data_noTBD["ALLOCATION AMOUNT"] = data_noTBD['ALLOCATION AMOUNT'].astype(int)
aggregated_data = aggregated_data.merge(data_noTBD.groupby(["Senate District", "Tax Allocation Year"]).sum()["ALLOCATION AMOUNT"].reset_index(), how="outer", on=["Senate District", "Tax Allocation Year"])

In [ ]:
# Pull senate district population for each year in the dataset
population_data = []
# Define request parameters

years = aggregated_data["Tax Allocation Year"].unique() # Years of interest
datasource = 'acs' # Survey name
subsource = 'acs5' # Subsurvey name
GET = 'B01001_001E' # Variables to query
FOR = 'state%20legislative%20district%20(upper%20chamber):*' # for predicate
IN = 'state:10'

# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Iterate over each non-nan year
for year in years:
    if year:
        # the acs2022 is not yet available, so use 2021 data
        if year == "2022":
            # Formatted API call
            data_url = f'https://api.census.gov/data/2021/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'  
        # Otherwise, use the population data for the year
        else:
            # Formatted API call
            data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

        # Read Census key into 'api_key'
        with open(keyfile) as key:
            api_key = key.read().strip()

        # Add key to url
        data_url = f'{data_url}&key={api_key}'

        # Request data and convert from json
        data = requests.get(data_url).json()
        # First entry in list is a list of variable names
        data = pd.DataFrame(data[1:], columns = data[0])
        # Add the year as a column
        data["year"] = year
        # Convert district number to an integer
        data["state legislative district (upper chamber)"] = data["state legislative district (upper chamber)"].astype(int).astype(str)
        # Drop the state column
        data.drop(columns="state", inplace=True)
        # Rename the population variable
        data.rename(columns={"B01001_001E": "Population"}, inplace=True)
        population_data.append(data)

population_data = pd.concat(population_data)

In [ ]:
# Add derived statistics, population data, senator name, and funding source to the aggregated dataframe
aggregated_data = aggregated_data.merge(gpd.read_file("data/aggregated_senate_new.json")[["name", "district"]], how="outer", left_on="Senate District", right_on="district").drop(columns="Senate District")
aggregated_data = aggregated_data.merge(population_data, left_on=["district", "Tax Allocation Year"], right_on=["state legislative district (upper chamber)", "year"], how="left").drop(columns=["state legislative district (upper chamber)", "year"])
# Filter out "S." from the beginning of senator names
aggregated_data["name"] = aggregated_data["name"].str.removeprefix("S. ")
# Add funding source
aggregated_data["Funding Source"] = "LIHTC"

# Calculate derived statistics
aggregated_data["Average Allocation per 100 Persons"] = aggregated_data["ALLOCATION AMOUNT"] * 100 / aggregated_data["Population"].astype(float)
aggregated_data["Average Population per Tax Credit Unit"] = aggregated_data["Population"].astype(float) / aggregated_data["# of Tax Credit Units"]
aggregated_data["Average Allocation per Tax Credit Unit"] = aggregated_data["ALLOCATION AMOUNT"].astype(float) / aggregated_data["# of Tax Credit Units"]

# Convert adjusted population to int. Assume anywhere with missing population numbers has a population of 0
aggregated_data["Population"] = aggregated_data["Population"].fillna(0).astype(int)

In [ ]:
# Melt the wide form data into long form data, grouping by district, senator name, funding source, and year
long_data = pd.melt(aggregated_data, id_vars=["district", "name", "Funding Source", "Tax Allocation Year"], value_vars=["Population", "# of Tax Credit Units", "ALLOCATION AMOUNT", "Average Allocation per 100 Persons", "Average Population per Tax Credit Unit", "Average Allocation per Tax Credit Unit"])

# Remove nan district from long form data
long_data = long_data.loc[~long_data["district"].isna()]

# Calculate the averages of each variable across all districts in each year
yearly_averages = long_data.loc[~long_data["Tax Allocation Year"].isna()].groupby(["variable", "Tax Allocation Year"]).mean(numeric_only=True).reset_index()
yearly_averages["Funding Source"] = "LIHTC"
yearly_averages["name"] = np.nan
yearly_averages["district"] = "District Average"

# Add the averages to the long form data
long_data = pd.concat([long_data, yearly_averages])
# Fill missing values with 0 for processing
long_data.fillna(0).to_csv("data/long_tax_data.csv", index=False)

In [ ]:
# Split district data into 2022 and non-2022 sections to isolate the 2020 senate district cycle
aggregated_data2010 = aggregated_data.loc[aggregated_data["Tax Allocation Year"] < "2022"]
aggregated_data2020 = aggregated_data.loc[aggregated_data["Tax Allocation Year"] >= "2022"]

In [ ]:
# Get current senator names for each district
extras = gpd.read_file("data/aggregated_senate_new.json")[["name", "district"]]
extras["district"] = extras["district"].astype(int).astype(str)

# Read trimmed senate distrcit shapes
trim2020 = gpd.read_file("data/2020Senate_Districts_Trimmed.geojson", driver="GeoJSON")
trim2020["district"] = trim2020["SLDUST"].astype(int).astype(str)
trim2020 = trim2020[["district", "geometry"]]

# Add derived statistics, population data, senator name, and funding source to the trimmed shapefiles
trim2020 = trim2020.merge(extras, how="left", on="district")
# Filter out "S." from the beginning of senator names
trim2020["name"] = trim2020["name"].str.removeprefix("S. ")
# Add population data and duplicate across years
trim2020 = trim2020.merge(population_data, left_on="district", right_on="state legislative district (upper chamber)", how="outer").drop(columns="state legislative district (upper chamber)")
trim2020 = trim2020.loc[trim2020["year"].isin(["2022"])]
# Add funding source
trim2020["Funding Source"] = "LIHTC"





# Read trimmed senate distrcit shapes
trim2010 = gpd.read_file("data/2010Senate_Districts_Trimmed.geojson", driver="GeoJSON")
trim2010["district"] = trim2010["SLDUST"].astype(int).astype(str)
trim2010 = trim2010[["district", "geometry"]]

# Add derived statistics, population data, senator name, and funding source to the trimmed shapefiles
trim2010 = trim2010.merge(extras, how="left", on="district")
# Filter out "S." from the beginning of senator names
trim2010["name"] = trim2010["name"].str.removeprefix("S. ")
# Add population data and duplicate across years
trim2010 = trim2010.merge(population_data, left_on="district", right_on="state legislative district (upper chamber)", how="outer").drop(columns="state legislative district (upper chamber)")
trim2010 = trim2010.loc[trim2010["year"].isin(["2016", "2017", "2018", "2019", "2020", "2021"])]
# Add funding source
trim2010["Funding Source"] = "LIHTC"

In [ ]:
# Drop district information from aggregated data
aggregated_data2010 = aggregated_data2010.drop(columns=["Population", "Funding Source", "name"])

# Attach senate districts to wide form aggregated data
aggregated_data2010 = aggregated_data2010.merge(trim2010, left_on=["district", "Tax Allocation Year"], right_on=["district", "year"], how="right").drop(columns="Tax Allocation Year")





# Drop district information from aggregated data
aggregated_data2020 = aggregated_data2020.drop(columns=["Population", "Funding Source", "name"])

# Attach senate districts to wide form aggregated data
aggregated_data2020 = aggregated_data2020.merge(trim2020, left_on=["district", "Tax Allocation Year"], right_on=["district", "year"], how="right").drop(columns="Tax Allocation Year")                                                                                                 

In [ ]:
# Re-attach data for summing
aggregated_data = pd.concat([aggregated_data2010, aggregated_data2020])

In [ ]:
# Compute the sum of all allocations in a district
sums = aggregated_data.groupby("district").sum(numeric_only=True).reset_index()
# Add name, adjusted population, funding source, and geometry back in
sums = aggregated_data2020[["district", "name", "Population", "Funding Source", "geometry"]].drop_duplicates("district").merge(sums, on="district", how="right")
# Label tax allocation year as sum over time
sums["year"] = "Sum over All Time"
# Add sums over time to aggregated data
aggregated_data2020 = pd.concat([aggregated_data2020, sums])

In [ ]:
# Print the wide form data for valid districts to a geojson
gpd.GeoDataFrame(aggregated_data2020.loc[~aggregated_data2020["district"].isna()]).to_file("data/aggregated_with_geo2020.geojson", driver="GeoJSON")
# Print the wide form data without geoometry to a csv
aggregated_data2020.drop(columns="geometry").to_csv("data/aggregated_data_with_na2020.csv", index=False)


# Print the wide form data for valid districts to a geojson
gpd.GeoDataFrame(aggregated_data2010.loc[~aggregated_data2010["district"].isna()]).to_file("data/aggregated_with_geo2010.geojson", driver="GeoJSON")
# Print the wide form data without geoometry to a csv
aggregated_data2010.drop(columns="geometry").to_csv("data/aggregated_data_with_na2010.csv", index=False)




Figure out how senate districts overlap with senate districts


NOTE: PyPDF2 needs to be added to the environment and imported to run this code

In [ ]:
# Download census block groups
blocks = requests.get("https://www2.census.gov/geo/tiger/GENZ2022/shp/cb_2022_10_bg_500k.zip")
blocks = gpd.read_file(BytesIO(blocks.content))
blocks

In [ ]:
lines = []

# creating a pdf file object
with open('data/CensusBlockBreakdownbySenateDistrict.pdf', 'rb') as pdfFileObj:
    # creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    # printing number of pages in pdf file
    print(pdfReader.numPages)

    # Iterate over each page
    for pageNum in range(pdfReader.numPages):
        # creating a page object
        pageObj = pdfReader.getPage(pageNum)
        
        # extracting text from page
        pageText = pageObj.extractText()
        
        # extract lines from each page
        pageLines = pageText.split("\n")
        
        for line in pageLines:
            lines.append(line.split(' '))

# Pull the columns out and separate columns that were incorrectly joined
columns = lines[0]
columns[1] = columns[0][5:] + columns[1]
columns[0] = columns[0][:5]
columns.insert(2, columns[2][:6])
columns[3] = columns[3][6:]
data = [x for x in lines if x[0].isdigit()]
for row in data:
    row.insert(1, row[0][2:])
    row[0] = row[0][:2]
    row.insert(2, row[2][:5])
    row[3] = row[3][5:]
    
senate_census_map = pd.DataFrame(data, columns=columns)
senate_census_map

In [ ]:
# Crosstabulate the connections between senate districts and census block groups
crosstab = pd.crosstab(senate_census_map["Proposed2022_SD"], senate_census_map["BlockGroup"])
# Get a list of census block groups that overlap with each senate district
senate_per_bg = crosstab.apply(lambda x : x[x != 0].index.values)
senate_per_bg.loc[senate_per_bg.apply(len) != 1]




In [ ]:
senate_per_bg2 = crosstab.apply(lambda x : x[x != 0].values)
senate_per_bg2.loc[senate_per_bg2.apply(len) != 1]

In [ ]:
scbg = pd.concat([senate_per_bg, senate_per_bg2], axis=1)
scbg.loc[(scbg.apply(lambda x : x.apply(len)) != 1).all(axis=1)].values

In [ ]:
scbg.columns = ["Senate Districts", "Blocks per District"]
scbg.to_csv("data/Senate_to_Block_Groups.csv")